In [ ]:
import os
import random

# Path to the directory containing the class folders
dataset_dir = "F:/Downloads/terrain_recognition/train"

In [ ]:
# Define the ranges for slipperiness and roughness values for each class
class_ranges = {
    "Grassy": {"slipperiness_min": 0.1, "slipperiness_max": 0.5, "roughness_min": 0.2, "roughness_max": 0.7},
    "Marshy": {"slipperiness_min": 0.3, "slipperiness_max": 0.7, "roughness_min": 0.4, "roughness_max": 0.8},
    "Rocky": {"slipperiness_min": 0.5, "slipperiness_max": 0.9, "roughness_min": 0.6, "roughness_max": 0.9},
    "Sandy": {"slipperiness_min": 0.2, "slipperiness_max": 0.6, "roughness_min": 0.1, "roughness_max": 0.5}
}


In [ ]:
# Initialize a dictionary to store image paths, class labels, slipperiness, and roughness values
image_data = []

In [ ]:
# Loop through each class folder
for class_folder, class_info in class_ranges.items():
    # Construct the full path to the class folder
    class_folder_path = os.path.join(dataset_dir, class_folder)
    
    # Check if it's a directory
    if os.path.isdir(class_folder_path):
        # List all files in the class folder
        image_files = os.listdir(class_folder_path)
        
        # Loop through each image file
        for image_file in image_files:
            # Construct the full path to the image file
            image_path = os.path.join(class_folder_path, image_file)
            
            # Extract the class label from the image path
            class_label = class_folder
            
            # Generate random slipperiness and roughness values within the specified range
            slipperiness = random.uniform(class_info["slipperiness_min"], class_info["slipperiness_max"])
            roughness = random.uniform(class_info["roughness_min"], class_info["roughness_max"])
            
            # Store the image data in a dictionary
            image_info = {
                "image_path": image_path,
                "class_label": class_label,
                "slipperiness": slipperiness,
                "roughness": roughness
            }
            
            # Append the image data to the list
            image_data.append(image_info)

In [ ]:
# Print the image data (image path, class label, slipperiness, and roughness)
for item in image_data:
    print("Image path:", item["image_path"])
    print("Class label:", item["class_label"])
    print("Slipperiness:", item["slipperiness"])
    print("Roughness:", item["roughness"])
    print()


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from PIL import Image
import os

In [ ]:
%pip install --upgrade tensorflow

In [ ]:
# Define constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 4  # Number of terrain classes (rocky, marshy, grassy, sandy)
EPOCHS = 50

In [ ]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)


In [ ]:
# Load training data from folders
train_generator = train_datagen.flow_from_directory(
    'F:/Downloads/terrain_recognition/train',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)


In [ ]:
# Load validation data from folders
validation_generator = train_datagen.flow_from_directory(
    'F:/Downloads/terrain_recognition/val',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
# Create the base MobileNetV3 model (without top layers)
base_model = MobileNetV3Large(input_shape=(*IMAGE_SIZE, 3), include_top=False)

In [ ]:
# Add custom top layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)

In [ ]:
# Create separate output layers for terrain classification, slipperiness, and roughness predictions
terrain_output = Dense(NUM_CLASSES, activation='softmax', name='terrain_output')(x)
slipperiness_output = Dense(1, activation='linear', name='slipperiness_output')(x)
roughness_output = Dense(1, activation='linear', name='roughness_output')(x)


In [ ]:
# Create the final model with multiple outputs
model = Model(inputs=base_model.input, outputs=[terrain_output, slipperiness_output, roughness_output])

In [ ]:
# Compile the model with appropriate loss functions and metrics for each output
model.compile(
    optimizer='adam',
    loss={'terrain_output': 'categorical_crossentropy', 'slipperiness_output': 'mean_squared_error', 'roughness_output': 'mean_squared_error'},
    metrics={'terrain_output': 'accuracy', 'slipperiness_output': 'mae', 'roughness_output': 'mae'}
)

In [ ]:
# Define the checkpoint directory and file naming
checkpoint_dir = './training_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, 'cp-{epoch:04d}.ckpt')


In [ ]:
# Create a callback to save model checkpoints every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq=5 * (len(train_generator) // BATCH_SIZE),  # Save every 5 epochs
    verbose=1
)

In [ ]:
# Load the weights if available (for resuming training)
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint:
    model.load_weights(latest_checkpoint)

In [ ]:
# Train the model with the checkpoint callback
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=[cp_callback]
)

In [ ]:
# Save the trained model
model.save('terrain_quantities_classification_model.h5')

In [ ]:
import numpy as np 
model = tf.keras.models.load_model('terrain_quantities_classification_model.h5')

# Load and preprocess a single test image
image_path = model = tf.keras.models.load_model('terrain_quantities_classification_model.h5')

# Load and preprocess a single test image
image_path = model = tf.keras.models.load_model('terrain_quantities_classification_model.h5')

# Load and preprocess a single test image
image_path = "F:/Downloads/terrain_recognition/test/Marshy/Marshy (2)_4_17.png"
image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
image = tf.keras.preprocessing.image.img_to_array(image)
image = np.expand_dims(image, axis=0)
image = image / 255.0  # Normalize the image

# Make predictions
predictions = model.predict(image)

# Separate the predictions for terrain classification, slipperiness, and roughness
terrain_predictions, slipperiness_predictions, roughness_predictions = predictions

# Print the results
print("Terrain Classification Predictions:")
terrain_labels = ['Rocky', 'Marshy', 'Grassy', 'Sandy']
terrain_class = terrain_labels[np.argmax(terrain_predictions)]
print(f"Terrain: {terrain_class}")

print("Slipperiness Prediction:")
slipperiness_value = slipperiness_predictions[0][0]
print(f"Slipperiness Value: {slipperiness_value}")

print("Roughness Prediction:")
roughness_value = roughness_predictions[0][0]
print(f"Roughness Value: {roughness_value}")
image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
image = tf.keras.preprocessing.image.img_to_array(image)
image = np.expand_dims(image, axis=0)
image = image / 255.0  # Normalize the image

# Make predictions
predictions = model.predict(image)

# Separate the predictions for terrain classification, slipperiness, and roughness
terrain_predictions, slipperiness_predictions, roughness_predictions = predictions

# Print the results
print("Terrain Classification Predictions:")
terrain_labels = ['Rocky', 'Marshy', 'Grassy', 'Sandy']
terrain_class = terrain_labels[np.argmax(terrain_predictions)]
print(f"Terrain: {terrain_class}")

print("Slipperiness Prediction:")
slipperiness_value = slipperiness_predictions[0][0]
print(f"Slipperiness Value: {slipperiness_value}")

print("Roughness Prediction:")
roughness_value = roughness_predictions[0][0]
print(f"Roughness Value: {roughness_value}")
image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
image = tf.keras.preprocessing.image.img_to_array(image)
image = np.expand_dims(image, axis=0)
image = image / 255.0  # Normalize the image

# Make predictions
predictions = model.predict(image)

# Separate the predictions for terrain classification, slipperiness, and roughness
terrain_predictions, slipperiness_predictions, roughness_predictions = predictions

# Print the results
print("Terrain Classification Predictions:")
terrain_labels = ['Rocky', 'Marshy', 'Grassy', 'Sandy']
terrain_class = terrain_labels[np.argmax(terrain_predictions)]
print(f"Terrain: {terrain_class}")

print("Slipperiness Prediction:")
slipperiness_value = slipperiness_predictions[0][0]
print(f"Slipperiness Value: {slipperiness_value}")

print("Roughness Prediction:")
roughness_value = roughness_predictions[0][0]
print(f"Roughness Value: {roughness_value}")